# **Dynamical system**
by: Edwin Saavedra C.

In [1]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd
from IPython.display import display

In [2]:
def stream(x,c,pi):
    dxdt =  pi[4]*x*c/(pi[1]+c)*(1-x) - pi[5]*x
    dcdt = -pi[0]*x*c/(pi[1]+c)*(1-x) + pi[2]*np.exp(-pi[3]*x)*(1-c)
    return dxdt,dcdt

xlin = np.linspace(0,2,51)
clin = np.linspace(0,1.2,51)
x,c   = np.meshgrid(xlin,clin)

In [3]:
parameters = [r'X_{\rm max}',r'C_\infty',r'K_C',r'\alpha',r'\beta',r'Y',r'k_{\rm die}',r'\hat{\mu}']
KSliders = [wd.FloatSlider(value=1,min=0,max=10,step=0.1,description=f'${k}$') for k in parameters]
piBoxes  = [wd.FloatText(description=fr'$\pi_{{{i}}}$',disabled=True,layout=wd.Layout(width="50%")) for i in range(6)]

def pi(change):
    piBoxes[0].value = KSliders[0].value/KSliders[1].value
    piBoxes[1].value = KSliders[2].value/KSliders[1].value
    piBoxes[2].value = KSliders[3].value/KSliders[7].value
    piBoxes[3].value = KSliders[4].value
    piBoxes[4].value = KSliders[5].value
    piBoxes[5].value = KSliders[6].value/KSliders[7].value
    
    for piBox in piBoxes: piBox.value = round(piBox.value,2)
    return None
    
for slider in KSliders:
    slider.observe(pi,'value')
    
pi(0)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/edsaac/dashboards/HEAD?urlpath=voila%2Frender%2FExponentialGrowth.ipynb)

In [4]:
eqTex0 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dC}{dt} &=& -\hat{\mu}X\dfrac{C}{K_C + C} + (C_\infty-C)\exp{\left(-\beta \dfrac{X}{X_{\rm max}}\right)} \\
    \dfrac{dX}{dt} &=& Y\hat{\mu}X\dfrac{C}{K_C + C} - k_{\rm die}X
\end{array}
\end{equation}''')

eqTex1 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    c &=& C/C_\infty \\
    x &=& X/X_{\rm max} \\
    \tau &=& \hat{\mu}t \\
    \\
    \pi_0 &=& X_{\rm max}/C_\infty \\
    \pi_1 &=& K_C/C_\infty \\
    \pi_2 &=& \alpha/\hat{\mu} \\
    \pi_3 &=& \beta \\
    \pi_4 &=& Y \\
    \pi_5 &=& k_{\rm die}/\hat{\mu}
\end{array}
\end{equation}''')

eqTex2 = wd.HTMLMath(r'''\begin{equation}
\begin{array}{rcl}
    \dfrac{dc}{d\tau} &=& -\pi_0 x\dfrac{c}{\pi_1 + c} + \pi_2(1-c)\exp{\left(-\pi_3 x\right)} \\
    \dfrac{dx}{d\tau} &=& \pi_4 x\dfrac{c}{\pi_1 + c} - \pi_5 x
\end{array}
\end{equation}''')

eqTexs = [eqTex0,eqTex1,eqTex2]

In [5]:
kw_streams = {'density':2,'linewidth':1,'arrowsize':0.5,
              'arrowstyle':'->','color':'darkmagenta','minlength':0.2}

output = wd.Output()

@output.capture(clear_output=True)
def plotStream(x,c,dx,dc):
    fig,ax = plt.subplots(figsize=[8,6],constrained_layout=True)
    ax.streamplot(x,c,dx,dc,**kw_streams)
    ax.grid(True)
    ax.set(ylabel="c",xlabel='x')
    plt.show();

def updateFig(b):
    piArray = [slider.value for slider in piBoxes]
    dx,dc = stream(x,c,piArray)
    plotStream(x,c,dx,dc);
    return None

accordion = wd.Accordion(eqTexs)
accordion.selected_index = None
accordion.set_title(0,'Governing ODEs')
accordion.set_title(1,'Nondimensionalization')
accordion.set_title(2,'Non-dimensional ODEs')

plotButton = wd.Button(description="Plot!",layout=wd.Layout(width="40%"))
plotButton.on_click(updateFig)

whiteSpace = wd.HTML("&darr;")

layoutBox = wd.Layout(align_items='center')
mainBox = wd.VBox([accordion,wd.HBox([wd.VBox(KSliders + [whiteSpace] + piBoxes + [whiteSpace,plotButton],layout=layoutBox),
                   output])])
display(mainBox)